In [ ]:
import json
import pyrender
import trimesh
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image, ImageDraw
from car_models import car_id2name, car_name2id
import cv2
import os
import sys
sys.path.append("../wlevel_estimator/")

fx=2304.54786556982
fy=2305.875668062
cx=1686.23787612802
cy=1354.98486439791

camera_matrix = np.array([[fx, 0,  cx],
                          [0,  fy, cy],
                          [0,  0,  1 ]], dtype=np.float32)

dist_coeffs = np.zeros((1,4))

In [ ]:
base_name = "171206_034625454_Camera_5"
the_car_id = 0

image_path = "D:\\Program\\Apollo\\instance\\train\\images\\"
pose_path = "D:\\Program\\Apollo\\instance\\train\\car_poses\\"

pil_image = Image.open(image_path+base_name+".jpg")
width, height = pil_image.size
np_image = np.array(pil_image)

with open(pose_path+the_file_name,"r") as file:
    car_poses = json.load(file)

car_pose = car_poses[the_car_id]
id = car_pose['car_id']
car_name = car_id2name[id].name
roll,pitch,yaw,x,y,z = car_pose['pose']

with open("../car_models_json/"+car_name+".json","r") as file:
    car_model = json.load(file)
    vertices = car_model['vertices']
    faces_array = np.array(car_model['faces']) - 1
    faces = faces_array.tolist()

In [ ]:
def eulerAnglesToRotationMatrix(theta) :
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])        
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])     
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])
    R = np.dot(R_z, np.dot( R_y, R_x ))

    return R

def createHomogenousMatrix(pitch,yaw,roll,x,y,z):
    R = eulerAnglesToRotationMatrix([pitch,yaw,roll])
    T = np.array([[x,y,z]])
    RT = np.hstack((R,T.T))
    return np.vstack((RT,[0.0,0.0,0.0,1.0]))